### <font color='white'>**Welcome to the Quantum ESPRESSO playground**</font>

In this notebook, we will learn how to run first principle density functional theory (DFT) calculations using the package Quantum ESPRESSO. Quantum ESPRESSO is a suite for electronic structure calculations and materials modeling at the nanoscale. You can check more information [here](https://www.quantum-espresso.org/).



### <font color='white'>1. Setting up Google Colab and clone the repo</font>

> Clone the repository to a local folder, if you did not do it yet. Alternatively download the necessary files from [here](https://github.com/TheoryGroupNanoGUNE/nanogune-winter-school-2026/tree/main/tutorial-2/silicon)

> If you run this notebook in Google Colab, you need to first install `conda` by executing this cell. Wait and check that the installation was succesful.

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

### <font color='white'>2. Installing Quantum ESPRESSO with conda</font>

> You need to install the Quantum ESPRESSO package. We will do this using `mamba`, which is similar to `conda` but a little bit faster. Execute the code and wait for the succesful installation.

In [ ]:
# install quantum espresso from conda
#!rm /usr/local/conda-meta/pinned
#!mamba install -y -c conda-forge quantum-espresso
!mamba install conda-forge::qe

> As of February 2026, the Quantum ESPRESSO version on  `conda` is 7.5. A test run can be done by calling the following command, which will tell you which version of the executable `pw.x` we have installed.

In [ ]:
!pw.x -v

### <font color='white'>3. Your first Quantum ESPRESSO calculation</font>

> We will now run a bandstructure calculation for bulk silicon. Before we can run a `bands` calculation, we need to perforn a single-point self consistent field calculation. We will go step by step.

> First, we create a directory called "Example_Si" using the terminal and navigate into it.

In [ ]:
import os
os.chdir('/content')
!pwd
!mkdir Example_Si
os.chdir('Example_Si')
!pwd

In [ ]:
!pwd

### <font color='white'>3.1 Input files</font>

> Now, we will obtain the files from the Github repository. We have 3 input files, which end with `.in`, a plot script and a pseudopotential file. Inspect those files and localize the data from the structure we want to use for the calculation.

> The pseudopotential is an efficientt way to describe the effects of the core electrons of an atom and its nucleus. There are different types of pseudopotentials that can be used in Quantum ESPRESSO, for example, check [here](https://www.quantum-espresso.org/pseudopotentials/). These files also have approximations for the density functional theory exchange correlation functional.

In [ ]:
#!rm *
#!rm -r *
# !wget https://github.com/TheoryGroupNanoGUNE/nanogune-winter-school-2026/tree/main/tutorial-2/silicon/Si.UPF
# Alternative: download locally and upload each file
from google.colab import files
uploaded = files.upload()
# Check that the files are there
!ls

### <font color='white'>Interlude: Understanding Density Functional Theory (DFT)</font>

> **The core idea**

In standard quantum mechanics, we describe electrons using a wavefunction ($\Psi$). However, for a system with $N$ electrons, the wavefunction depends on $3N$ variables, making it impossible to solve for large systems. DFT simplifies this by proving that all ground-state properties are determined by the **electron density** $n(\mathbf{r})$, which only depends on **3 variables** ($x, y, z$), regardless of the number of electrons.


> **The total energy functional**

The total energy of the system is written as a functional of the density function

\begin{equation}
E[n(\mathbf{r})] = T[n(\mathbf{r})] + E_{\text{ext}}[n(\mathbf{r})] + E_{H}[n(\mathbf{r})] + E_{xc}[n(\mathbf{r})]
\end{equation}

where

*   $T[n(\mathbf{r})]$: kinetic energy of the electrons.

*   $E_{ext}[n(\mathbf{r})]$: interaction with the atomic nuclei.

*   $E_{H}[n(\mathbf{r})]$: Hartree energy (classical electron-electron repulsion).

*   $E_{xc}[n(\mathbf{r})]$: exchange-correlation energy (quantum mechanical effects).

In practice, minimization of this function with respect to the density yields the Kohn-Sham equations.

> **The self-consistent field (SCF) cycle**

Quantum ESPRESSO solves this by starting with a "guess" of the density and refining it in a loop.

* Guess the initial electron density $n(\mathbf{r})$.

* Solve the Kohn-Sham equations to find new wavefunctions.

* Calculate the new density from those wavefunctions: $n(\mathbf{r}) = \sum |\phi_i|^2$.

* Repeat until the energy stops changing (convergence)

Since the exact mathematical form of the exchange-correlation functional ($E_{xc}$) is unknown, it must be approximated.

What approximation are we using here?

### <font color='white'>3.2 Run a SCF calculation</font>

> First, run a calculation to converge the density in a self-consistent fashion. Inspect and understand the command below before running.

In [ ]:
!mpirun --allow-run-as-root --oversubscribe -np 2 --host localhost:2 pw.x < pw.in | tee pw.scf.out

> Inspect the resulting output file using the command `cat`.

* Have your calculations converged to a fixed point?

* Locate the number of iterations, the total energy and the SCF accuracy.

* What is the Fermi energy?

**Note on the basis set**
> In Quantum ESPRESSO, the wavefunctions are expanded using a plane wave  basis set. Instead of using complex atomic orbitals, the code represents electrons as a sum of simple periodic waves (like a 3D Fourier series). This is mathematically convenient for crystals but requires a way to decide how many waves are "enough." This is done through the parameters `ecutwfc` y `ecutrho`.




In [ ]:
# !cat pw.scf.out
# !grep 'highest' pw.scf.out

### <font color='white'>3.3 Run a non-SCF calculation</font>

> Once the first SCF calculation has converged, we must run a non-SCF calculation to obtain the final wavefunctions and energies. This run will use the ground state electron density, Hartree, exchange and correlation potentials obtained in the previous step. Run this calculation using the following command, previously inspecting the input file with the command `cat`.


* Inspect the parameter `nbnd`. Based on the number of electrons in your system, is this enough to see the band gap?

* Look at the **k**-path. How many k-points are we considering? Check if this is consistent with you output. Which of this points is the origin of the Brillouin zone (in the reciprocal space)

In [ ]:
!mpirun --allow-run-as-root --oversubscribe  -np 2 --host localhost:2  pw.x < pw_nscf.in | tee pw.nscf.out

### <font color='white'>3.4 Postprocessing</font>

> The non-SCF calculation provides the energies and wavefunctions along a predefined path in the Brillouin zone. For the final plot, we still need to postprocess the bands with the `bands.x` executable.

In [ ]:
!mpirun --allow-run-as-root --oversubscribe  -np 2 --host localhost:2 bands.x < bands.in | tee bands.out

### <font color='white'>3.5 Plotting the bands</font>

> Finally, plot the silicon bandstructure. You can do it with the script of your choice or with the `plotband.x` utility.

> A **bandstructure** plot represents the dispersion relation of electrons in a solid, showing how the energy $E$ of an electron changes as a function of its crystal momentum or **k**-vector. By calculating $E$ for many values of $\mathbf{k}$ along high-symmetry paths (such as $L \rightarrow \Gamma \rightarrow X \rightarrow U \rightarrow \Gamma$), we can determine the electronic nature of the material:

* Metals: The highest occupied band and the lowest unoccupied band overlap, or the Fermi energy intersects one or more bands. There is no energy gap.

* Semiconductors and insulators: A finite energy gap ($E_g$​) exists between the Valence Band Maximum (VBM) and the Conduction Band Minimum (CBM).

    - Direct gap: The VBM and CBM occur at the same k-point.

    - Indirect gap: The VBM and CBM occur at different k-points

In [ ]:
!plotband.x
# Use the following parameters when asked for the prompt.
# name: si_bands.dat
# Emin, Emax = -6, 16
# output file (gnuplot): si_bands.gnuplot
# output file (ps): si_bands.ps
# Efermi: -6.37
# DeltaE, reference: 1,1

> Download the resulting bandstructure and check your results.

* What is the bandgap predicted for your structure?

* Is bulk silicon a semiconductor?

In [ ]:
!apt-get install ghostscript -y
!ps2pdf si_bands.ps si_bands.pdf
from google.colab import files
files.download('si_bands.pdf')  # Replace with your file name

### <font color='white'>4. Exercise: graphene bandstructure</font>

> Repeat the previous steps using the input files provided. Find the appropiate pseudopotentials and substitute the values `???` in the input files. Can you obtain the bandgap of graphene?

(credits: tutorial made by Daniel Hernangómez Pérez, inspired by the Quantum ESPRESSO examples and summer schools exercises)